# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robertsa, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import signal


# Obrazki
if not os.path.exists("jet.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png --no-check-certificate
if not os.path.exists("kw.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png --no-check-certificate
if not os.path.exists("moon.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png --no-check-certificate
if not os.path.exists("lenaSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png --no-check-certificate
if not os.path.exists("lena.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png --no-check-certificate
if not os.path.exists("plansza.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png --no-check-certificate

plansza = cv2.imread("plansza.png", cv2.IMREAD_GRAYSCALE)

filtr = np.ones((3, 3))
filtered = cv2.filter2D(plansza, -1, filtr)

modulo = abs(filtered-plansza)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(plansza, cmap='gray')
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Filtered CV2')

ax[2].imshow(modulo, cmap='gray')
ax[2].axis('off')
ax[2].set_title('Difference')

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
sizes = [3, 5, 9, 15, 35]

size = 4
fig, ax = plt.subplots(2, 3, figsize=(3*size, 2*size))

ax[0][0].imshow(plansza, cmap='gray')
ax[0][0].axis('off')
ax[0][0].set_title('Original')

for i, size_i in enumerate(sizes):
    filtr = np.ones((size_i, size_i))
    filtr = filtr/sum(sum(filtr))
    filtered = cv2.filter2D(plansza, -1, filtr)

    i += 1
    ax[i//3][i%3].imshow(filtered, cmap='gray')
    ax[i//3][i%3].axis('off')
    ax[i//3][i%3].set_title(f'Window size: {size_i}')

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
lena = cv2.imread("lena.png")

filter = np.ones((3, 3)) / 9

filtered_lena = cv2.filter2D(lena, -1, filter)

modulo_lana = abs(filtered_lena-lena)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered_lena)
ax[1].axis('off')
ax[1].set_title('Filtered CV2')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
M = np.array([[1, 2, 1],
              [2, 4, 2],
              [1, 2, 1]])
M = M/sum(sum(M))

filtered_lena = cv2.filter2D(lena, -1, M)

modulo_lana = abs(filtered_lena-lena)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered_lena)
ax[1].axis('off')
ax[1].set_title('Filtered CV2')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr `odchylenie standardowe` wpływa na `kształt` filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()
    

odchylenia = [0.5, 1, 5]

lena = cv2.imread("lena.png")

for i, odchyl in enumerate(odchylenia):
    M = fgaussian(5, odchyl)
    mesh(M, 5)
    filtered_lena = cv2.filter2D(lena, -1, M)
    fig, ax = plt.subplots()
    ax.imshow(filtered_lena, cmap='gray')
    ax.axis('off')
    ax.set_title(f"Gaussina blur odchylenie: {odchyl}")


### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję `cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
lena_szum = cv2.imread("lenaSzum.png")

filter = np.ones((3, 3)) / 9

filtered_lena = cv2.filter2D(lena_szum, -1, filter)
modulo_lana = abs(filtered_lena-lena_szum)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena_szum)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered_lena)
ax[1].axis('off')
ax[1].set_title('Filtr uśredniający')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

filtered_lena = cv2.medianBlur(lena_szum, 3)
modulo_lana = abs(filtered_lena-lena_szum)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena_szum)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered_lena)
ax[1].axis('off')
ax[1].set_title('Filtr medianowy')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
filter = np.ones((3, 3)) / 9

filtered_lena = cv2.filter2D(lena, -1, filter)
modulo_lana = abs(filtered_lena-lena)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered_lena)
ax[1].axis('off')
ax[1].set_title('Filtr uśredniający')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

filter = np.ones((3, 3)) / 9

filtered_lena = cv2.medianBlur(lena, 3)
modulo_lana = abs(filtered_lena-lena)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(filtered_lena)
ax[1].axis('off')
ax[1].set_title('Filtr medianowy')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt `x`, `+`), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z `x` i mediana z `+`,
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
image = cv2.imread("lena.png")

for i in range(10):
    image = cv2.medianBlur(image, 5)

modulo_lana = abs(image-lena)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(lena)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(image)
ax[1].axis('off')
ax[1].set_title('Posteryzacja')

ax[2].imshow(modulo_lana)
ax[2].axis('off')
ax[2].set_title('Difference')

## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\tag{1}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez sumę wag dodatnich (ewentualnie sumę modułów wszystkich wag).
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodanie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
moon = cv2.imread("moon.png")

M = np.array([[0, 1, 0],
              [1, -4, 1],
              [0, 1, 0]])
M = M/ sum(sum(abs(M)))

filtered_moon = cv2.filter2D(moon.astype(np.int16), -1, M)

normalized_plus = filtered_moon + 128
normalized_modulo = abs(filtered_moon)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(moon, cmap='gray', vmin=0, vmax=255)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(normalized_plus,cmap='gray', vmin=0, vmax=255)
ax[1].axis('off')
ax[1].set_title('Dodanie 128')

ax[2].imshow(normalized_modulo,cmap='gray', vmin=0, vmax=255)
ax[2].axis('off')
ax[2].set_title('Moduł')

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
moon = moon.astype(np.int16)

plus = moon + filtered_moon
minus = abs(moon - filtered_moon)

size = 4
fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
ax[0].imshow(moon.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(plus.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
ax[1].axis('off')
ax[1].set_title('Plus')

ax[2].imshow(minus.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
ax[2].axis('off')
ax[2].set_title('Minus')

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\tag{2}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\tag{3}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie `prostopadłe`):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\tag{4}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
def krawedzie(image, M):
    filtered = cv2.filter2D(image.astype(np.int16), -1, M)

    normalized_plus = filtered + 128
    normalized_modulo = abs(filtered)

    size = 4
    fig, ax = plt.subplots(1, 3, figsize=(3*size, size))
    ax[0].imshow(image, cmap='gray', vmin=0, vmax=255)
    ax[0].axis('off')
    ax[0].set_title('Original')

    ax[1].imshow(normalized_plus.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
    ax[1].axis('off')
    ax[1].set_title('Skalowanie')

    ax[2].imshow(normalized_modulo.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
    ax[2].axis('off')
    ax[2].set_title('Moduł')

kw = cv2.imread("kw.png")
jet = cv2.imread("jet.png")

R1 = np.array([[0, 0, 0],
              [-1, 0, 0],
              [0, 1, 0]])
R1 = R1/ sum(sum(abs(R1)))

R2 = np.array([[0, 0, 0],
              [0, 0, -1],
              [0, 1, 0]])
R2 = R2/ sum(sum(abs(R2)))
krawedzie(kw, R1)
krawedzie(kw, R2)

P1 = np.array([[-1, 0, 1],
              [-1, 0, 1],
              [-1, 0, 1]])
P1 = P1/ sum(sum(abs(P1)))

P2 = np.array([[-1, -1, -1],
              [0, 0, 0],
              [1, 1, 1]])
P2 = P2/ sum(sum(abs(P2)))
krawedzie(kw, P1)
krawedzie(kw, P2)

S1 = np.array([[-1, 0, 1],
              [-1, 0, 1],
              [-1, 0, 1]])
S1 = S1/ sum(sum(abs(S1)))

S2 = np.array([[-1, -1, -1],
              [0, 0, 0],
              [1, 1, 1]])
S2 = S2/ sum(sum(abs(S2)))
krawedzie(kw, S1)
krawedzie(kw, S2)
krawedzie(jet, S1)
krawedzie(jet, S2)

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\tag{5}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
def filtr_kombinowany(image):
    S1 = np.array([[-1, 0, 1],
              [-1, 0, 1],
              [-1, 0, 1]])
    S1 = S1/ sum(sum(abs(S1)))

    S2 = np.array([[-1, -1, -1],
                  [0, 0, 0],
                  [1, 1, 1]])
    S2 = S2/ sum(sum(abs(S2)))

    o1 = cv2.filter2D(image, -1, S1).astype(np.int16)
    o2 = cv2.filter2D(image, -1, S2).astype(np.int16)

    ow = np.sqrt(o1**2 + o2**2).astype(np.int16)

    size = 4
    fig, ax = plt.subplots(1, 2, figsize=(2*size, size))
    ax[0].imshow(image.astype(np.uint8), cmap='gray', vmin=0, vmax=255)
    ax[0].axis('off')
    ax[0].set_title('Original')

    ax[1].imshow(ow.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
    ax[1].axis('off')
    ax[1].set_title('Filtr kombinowany')

filtr_kombinowany(kw)

6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
def filtr_kombinowany_v2(image):
    S1 = np.array([[-1, 0, 1],
              [-1, 0, 1],
              [-1, 0, 1]])
    S1 = S1/ sum(sum(abs(S1)))

    S2 = np.array([[-1, -1, -1],
                  [0, 0, 0],
                  [1, 1, 1]])
    S2 = S2/ sum(sum(abs(S2)))

    o1 = cv2.filter2D(image, -1, S1).astype(np.int16)
    o2 = cv2.filter2D(image, -1, S2).astype(np.int16)

    ow = abs(o1) + abs(o2).astype(np.int16)

    size = 4
    fig, ax = plt.subplots(1, 2, figsize=(2*size, size))
    ax[0].imshow(image.astype(np.uint8), cmap='gray', vmin=0, vmax=255)
    ax[0].axis('off')
    ax[0].set_title('Original')

    ax[1].imshow(ow.astype(np.uint8),cmap='gray', vmin=0, vmax=255)
    ax[1].axis('off')
    ax[1].set_title('Filtr kombinowany v2')

filtr_kombinowany_v2(kw)


7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
filtr_kombinowany(jet)
filtr_kombinowany_v2(jet)